In [1]:
#%pip install tensorflow

In [22]:
# built-in modules
import os
import pickle
import random
import string   

# third-party modules
import biosppy as bp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
from scipy import signal
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


# local modules
import multiAE_creation as mc
from utils_classification import fus_classifier

# files names
bitalino = "/Users/saraiva/dev/Respiration/APNEIA/"
# all_files = [folder+ol for ol in os.listdir(folder)]
bitalino_files = [bitalino + ol for ol in os.listdir(bitalino)]
print(bitalino_files)

['/Users/saraiva/dev/Respiration/APNEIA/T05_6_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T03_7_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T05_1_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T02_1_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T02_6_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T03_6_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T05_7_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T03_1_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T04_1_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T02_7_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T04_6_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T06_6_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T06_1_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T01_1_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T01_6_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/T06_7_BITALINO.csv', '/Users/saraiva/dev/Respiration/APNEIA/

# Load labels

all_ap will contain the labels of the 70 subjects, while resp_ap will contain the labels of the 8 subjects which have both ecg and resp signals

pd_all and pd_resp show the number of apnea and normal samples for all subjects

In [ ]:
from scipy import signal
def normal(sg, minsg, maxsg):
    res = 100 * (sg - minsg) / (maxsg - minsg)
    return res

def find_extremes(sig, mode, th):
    indexes, values = bp.tools.find_extrema(sig, mode)
    ind, peaks = [], []
    for pk in range(len(values)):
        if abs(values[pk] - np.mean(sig)) > float(th):
            ind += [indexes[pk]]
            peaks += [values[pk]]
    return ind, peaks


def deep_breath(sig):
    #the signal of the first apnea is received. The deep breath is characterized by a
    #sudden increase, which corresponds to a high value of the positive derivative.
    #to find the deep breathe we use find extremes
    ind, peaks = find_extremes(sig, 'both', 0.3)
    if peaks == []:
        plt.plot(sig)
        plt.scatter(ind,peaks)
        plt.show()
    div1 = np.diff(peaks)
    deep_b = np.max(div1)
    if deep_b > 0:
        print('\ndeep breathe was found')
        deep_idx = int(np.where(div1 == deep_b)[0])
        res = [peaks[deep_idx], peaks[deep_idx+1]]
    else:
        print(peaks, div1)
        res = 0
    print(res)
    return res

marker_label = dict([(2, 'N'), (5, 'N'), (8, 'A'), (10, 'A'), (12, 'A'), (14, 'A'), (16, 'A')])
_label = dict([(0, 'N'), (1, 'N'), (2, 'A'), (3, 'A'), (4, 'A'), (5, 'A'), (6, 'A')])

sb.set(font_scale=3)
user_labs = []
user_data = []
for bf in range(len(bitalino_files)):
    labs = []
    data = []
    df = pd.read_csv(bitalino_files[bf], parse_dates=True, index_col=0, header=0,sep=';')

    resp = np.array(bp.tools.filter_signal(df.A1, ftype='butter', band='bandpass', order=2, frequency=[0.01,0.35],sampling_rate=1000.)[
                        'signal'])
    resp = normal(resp,np.min(resp),np.max(resp))
    list_markers = [[mk, marker] for mk, marker in enumerate(df.MARKER) if
                        marker != 0 and marker in marker_label.keys()]

    print(list_markers)
    arr = np.array(list_markers)[:,0]*0.001
    la = 0
    for o, lm in enumerate(arr[::2]):
        print(o, lm)
        init = list_markers[o*2][0]
        end = list_markers[o*2+1][0]
        if _label[la] == 'A':
            x_arr = np.arange(init*0.001,end*0.001, 0.001)
            if len(np.arange(init*0.001,end*0.001, 0.001)) != len(resp[init:end]):
                x_arr = x_arr[:-1]
        la+=1


    deep = resp[list_markers[4][0]:list_markers[5][0]]
    xminmax = deep_breath(deep)

    for lm in range(0,len(list_markers),2):
        sig = resp[list_markers[lm][0]:list_markers[lm+1][0]]
        lab = marker_label[list_markers[lm][1]]
        
        for k in range(0,len(sig)-60000,60000):
            signl = sig[k:k+59999]
            signl = signal.resample(normal(signl, np.min(signl), np.max(signl)),1000)
            labs += [lab]
            data += [signl]

    user_labs += [labs]
    user_data += [data]
            

        



In [5]:
with open(r"user_labs", "wb") as output_file:
    pickle.dump(user_labs, output_file)

with open(r"user_data", "wb") as output_file:
    pickle.dump(user_data, output_file)

In [6]:

# _UR = data_resp
#_ap = pickle.load(open('RespN_Y','rb'))
_UR = user_data
_ap = user_labs

# 2 autoencoders

In [9]:

def part_balance(list_vecs, list_lab):
    train = []
    lab = []
    for v, vec in enumerate(list_vecs):

        num = list_lab[v].count('A')
        if num != 0:
            sel = random.choices(vec[:-num], k=num)
            tr = np.concatenate([sel, vec[-num:]])
            ba = ['N']*num + ['A']*num

            train += [tr]
            lab += [ba]

            if len(tr)!=len(ba):
                aaaa
    return train, lab

def create_autoencoder(x_train,y_train, x_test, y_test, label, loss, 
                       activ = 'tanh', opt = 'adam', nodes = [500,250,50],epochs=100):
    mc.autoencoder_params(nodes[-1], 1000, nodes, activ, opt,  loss,
                          x_train,y_train, x_test, y_test,label, epochs)

def separate_classes(vecs,labs, classes=['A','N']):
    #take the whole set and separate classes.
    new_vec, new_labs = [],[]
    for cl in classes:
        
        cl_index = [i for i in range(len(labs)) if labs[i] == cl]
        vec_ = np.array(vecs)[cl_index]
        lab_ = np.array(labs)[cl_index]
        new_vec += [vec_]
        new_labs += [lab_]
    return new_vec, new_labs

# for user in range(len(_UR)):
    
#     training_set_x = list(_UR)
#     del(training_set_x[user])
#     training_set_y = _ap[:]
#     del(training_set_y[user])
#     training_set_x = np.concatenate(training_set_x, axis=0)
#     print(np.array(training_set_x).shape)
#     training_set_y = np.concatenate(training_set_y)
    
#     train, y_train = separate_classes(training_set_x, training_set_y)
    
#     ap_train = train[0]
#     n_train = train[1]
#     #training_set_y = np.array([1 if yt == 'N' else 0 for yt in training_set_y])
    
#     ap_train, ap_test, ap_ytrain, ap_ytest = \
#         train_test_split(ap_train, y_train[0], test_size=.4, random_state=42)
#     n_train, n_test, n_ytrain, n_ytest = \
#         train_test_split(n_train, y_train[1], test_size=.4, random_state=42)
#     #print('This train contains ' + str(list(ae_ytrain).count('A')) + ' Apneas and ' + str(list(ae_ytrain).count('N')) + ' Normal events')
    
#     loss = 'cosine_proximity'
#     activ='tanh'
#     opt='adam'
    
#     nodes = [500,250,50]
    
    
#     create_autoencoder(n_train, n_train, n_test, n_test, 'RN', loss, activ, opt, nodes = nodes, epochs=50)
#     #create_autoencoder(n_train, n_train, n_test, n_test, 'RC', loss, activ, opt, nodes = nodes, epochs=50)
    
#     encoder = pickle.load(open('RN_encoder', 'rb'))
#     decoder = pickle.load(open('RN_decoder', 'rb'))
    
#     testing_set_x = _UR[user]
#     testing_set_y = _ap[user]
    
#     enc = encoder.predict(testing_set_x)
#     dec = decoder.predict(enc)
    
#     for d in range(len(dec)):
        
#         plt.plot(dec[d], label=testing_set_y[d])
#         corr = [pearsonr(dec[d][i:i+100],testing_set_x[d][i:i+100])[0] for i in range(0,len(dec[d]),100)]
#         plt.title(corr)
#         plt.plot(testing_set_x[d], label='input')
#         plt.legend()
#         plt.show()
        

# Autocorrelation Autoencoder

In [ ]:


signal = _UR[:]
#all_corr = []
for user in range(len(signal)):
    
    training_set_x = list(signal)
    del(training_set_x[user])
    training_set_y = _ap[:]
    del(training_set_y[user])
    training_set_x = np.concatenate(training_set_x, axis=0)
    print(np.array(training_set_x).shape)
    training_set_y = np.concatenate(training_set_y)
    
    train, y_train = separate_classes(training_set_x, training_set_y)
    
    #ap_train = train[0]
    #n_train = train[1]
    #training_set_y = np.array([1 if yt == 'N' else 0 for yt in training_set_y])
    
    #ap_train, ap_test, ap_ytrain, ap_ytest = \
     #   train_test_split(ap_train, y_train[0], test_size=.4, random_state=42)
    n_train, n_test, n_ytrain, n_ytest = \
        train_test_split(train[1], y_train[1], test_size=.4, random_state=42)
    #print('This train contains ' + str(list(ae_ytrain).count('A')) + ' Apneas and ' + str(list(ae_ytrain).count('N')) + ' Normal events')
    
    loss = 'cosine_similarity'
    #loss = 'mean_squared_error'
    activ='tanh'
    opt='adam'
    
    nodes = [500,250,50]
    
    
    #create_autoencoder(ap_train, ap_train, ap_test, ap_test, 'RN', loss, activ, opt, nodes = nodes, epochs=50)
    create_autoencoder(n_train, n_train, n_test, n_test, 'RN', loss, activ, opt, nodes = nodes, epochs=50)
    
    encoder = pickle.load(open('RN_encoder', 'rb'))
    decoder = pickle.load(open('RN_decoder', 'rb'))
    
    testing_set_x = np.array(signal[user])
    testing_set_y = _ap[user]
    
    enc = encoder.predict(testing_set_x)
    dec = decoder.predict(enc)
    save_corr = []
    enc_train = encoder.predict(training_set_x)
    dec_train = decoder.predict(enc_train)
    train_cl,test_cl = [],[]
    y_train_new = []
    y_test_new = []
    for d in range(len(dec_train)):
        corr_train = [pearsonr(dec_train[d][i:i+100],training_set_x[d][i:i+100])[0] for i in range(0,len(dec_train[d]),100)]
        if np.isfinite(corr_train).all():
            train_cl += [corr_train]
            y_train_new += [training_set_y[d]]

    for d in range(len(dec)):
        
        plt.plot(dec[d])
        corr = [pearsonr(dec[d][i:i+100],testing_set_x[d][i:i+100])[0] for i in range(0,len(dec[d]),100)]
        
        if np.isfinite(corr).all():
            test_cl += [corr]
            y_test_new += [testing_set_y[d]]
        
    cl_score = []
    
    names = ["SVM", "Random Forest"] 
    
    classifiers = [SVC(gamma=2, C=1, probability=True),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)]
    
    cl_score = fus_classifier(classifiers, names, train_cl,\
                                  test_cl, np.array(y_train_new), np.array(y_test_new), show=True)
    
        

In [14]:
cl_score

86.67

In [122]:
svm = [93,40,67,100,40,100,67,100,80,93,100,100,100,100,87,67,80,73,73,53,\
       100,100,93,80,100,100,100,73,100,100,93,100,87,80,87,93,100,80,73,100]
rf = [93,47,67,87,47,100,80,100,67,93,93,93,93,100,93,60,80,73,80,60,100,\
      100,93,80,100,100,100,73,93,93,80,93,87,80,87,87,100,73,60,80,80,93,47,87]

print(np.mean(svm))
print(np.std(svm))

print(np.mean(rf))
print(np.std(rf))



86.3
16.459343850834394
83.45454545454545
15.173922824513692


In [16]:
from scipy.stats import pearsonr
for data in user_data[0]:
    enc = encoder.predict(data)
    dec = decoder.predict(enc)


    for da in range(len(data)):
        
        plt.plot(data[da], label='input')
        plt.plot(dec[da], label='output')
        corr = pearsonr(data[da],dec[da])

        plt.title(str(corr))
        plt.legend()
        plt.show()

InvalidArgumentError: Graph execution error:

Detected at node model_139/dense_276/MatMul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/saraiva/dev/PreEpiSeizuresCode/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/saraiva/dev/PreEpiSeizuresCode/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/Users/saraiva/dev/PreEpiSeizuresCode/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 728, in start

  File "/Users/saraiva/dev/PreEpiSeizuresCode/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/V

# Classification Phase

In [8]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)


In [23]:

# names = ["Decision Tree", "AdaBoost","GradBoost"]

# classifiers = [DecisionTreeClassifier(max_depth=5),
#     AdaBoostClassifier(),GradientBoostingClassifier()]
#names = ["GradBoost"]
#classifiers = [GradientBoostingClassifier()]
names = ["KNN", "SVM", "Decision Tree", "Random Forest", "Neural Net",\
        "Adaboost", "GradBoost", "Naive Bayes", "QDA"]
classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1, probability=True),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

#names = ["SVM", "Random Forest", "NeuralNet"]

def choose_classifier(data, labels):
    best=0
    cl_train, cl_test, cl_ytrain, cl_ytest = \
        train_test_split(data, labels, test_size=.4, random_state=42)
    all_scores = []
    for n, c in zip(names, classifiers):
        print(n)
        scores = cross_val_score(c, data, labels, scoring='f1',cv=10)
        print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

        # Train the classifier
        c.fit(cl_train, cl_ytrain.ravel())

        # Predict test data
        y_test_predict = c.predict(cl_test)

        # Get the classification accuracy
        print("Accuracy: " + str(scores.mean()) + '%')
        print('-----------------------------------------')
        if scores.mean() > best:
            best_classifier = n
            best = scores.mean()
        all_scores+=[np.round(scores.mean()*100,2)]
        
    print('******** Best Classifier: ' + str(best_classifier) + ' ********')
    print(all_scores)
    return pd.DataFrame([all_scores], columns=names)


def score_classifier(c, n, train, test, y_train, y_test, show=False):
    all_scores = []

    # Train the classifier
    c.fit(train, y_train.ravel())

    # Predict test data
    y_pred = c.predict(test)
    
    #score = f1_score(y_test, y_pred, average='micro')
    score = accuracy_score(y_test,y_pred)
    
    # Get the classification accuracy
    print(str(n) + " --- Accuracy: " + str(score) + '%')
    print('-----------------------------------------')
    class_names = np.array(['Apnea', 'Normal'])
    if show:
        
        # Plot non-normalized confusion matrix
        plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')
        plt.show()
        
    #return [np.round(score*100,2), np.round(accuracy*100,2)]
    return np.round(score*100,2)

def fus_classifier(cl, ns, train, test, y_train, y_test, show=False):
    all_scores = []
    
    # Train the classifier
    y_pred = []
    
    cl[0].fit(train, y_train.ravel())
    cl[1].fit(train, y_train.ravel())
    proba1 = cl[0].predict_proba(test)
    proba2 = cl[1].predict_proba(test)
    #prob = [proba1[pb] if np.argmax([proba1[pb], proba2[pb]]) in [0,1] else proba2[pb] for pb in range(len(proba1))]
    prob= [[np.mean([proba1[i,0],proba2[i,0]]),np.mean([proba1[i,1],proba2[i,1]])] for i in range(len(proba1))]

    
    print(prob)
    y_pred = [np.argmax(prob[i]) for i in range(len(prob))]
    y_test = [1 if yt == 'N' else 0 for yt in y_test]
    print(y_test, y_pred)
    score = accuracy_score(y_test,y_pred)
    
        
    #print(c.predict_proba(test))
    # Get the classification accuracy
    print(str(ns) + " --- Accuracy: " + str(score) + '%')
    print('-----------------------------------------')
    class_names = np.array(['Apnea', 'Normal'])
    if show:
        
        # Plot non-normalized confusion matrix
        plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')

        plt.show()
        
        
    
    #return [np.round(score*100,2), np.round(accuracy*100,2)]
    return np.round(score*100,2)



def fusion_classifier(classifiers, names, train, test, y_train, y_test, show=False):
    all_scores = []
    
    cl_score = []
    
    for n, c in zip(names, classifiers):
    
        c.fit(train, y_train.ravel())
        
        prob = c.predict_proba(test)
        
        if n == names[0]:
            predictions = prob[0]
        else:  
            predictions = np.column_stack((np.maximum(predictions[0][:,0],prob[0][:,0]), np.maximum(predictions[0][:,1],prob[0][:,1])))

    y_pred = [np.argmax(np.array(yi)) for yi in predictions]

    print(y_pred)
    
    score = f1_score(y_test, y_pred, average='micro')
    
    #accuracy = accuracy_score(y_test,y_pred)
    # Get the classification accuracy
    print(" Fusion --- F1 Score: " + str(score) + '%')
    print('-----------------------------------------')
    class_names = np.array(['Apnea', 'Normal'])
    
    if show:
        
        # Plot non-normalized confusion matrix
        plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')

        # Plot normalized confusion matrix
        plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

        plt.show()      
        
    #return [np.round(score*100,2), np.round(accuracy*100,2)]
    return np.round(score*100,2)



# 30 Folds 

Classifier is a fusion from SVM with Random Forest

Autoencoder gives correlation values



In [24]:
from sklearn.model_selection import RepeatedKFold

Y = np.array(user_labs)
FS_X = np.array(user_data)
FS_X_train = [0]*30
FS_X_test = [0]*30
y_test = [0]*30
y_train = [0]*30
fold = 0

kf = RepeatedKFold(n_splits=10, n_repeats = 3,random_state=10)

for train_index, test_index in kf.split(Y):
    print(Y.shape)
    print(test_index, train_index)
    FS_X_train[fold] = np.vstack(FS_X[train_index])
    y_train[fold] = np.concatenate(Y[train_index], axis=0)
    FS_X_test[fold] = np.vstack(FS_X[test_index])
    y_test[fold] = np.concatenate(Y[test_index], axis=0)
    fold +=1


(45, 15)
[10 20 30 33 34] [ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 21 22 23 24 25
 26 27 28 29 31 32 35 36 37 38 39 40 41 42 43 44]
(45, 15)
[ 3  7 18 23 40] [ 0  1  2  4  5  6  8  9 10 11 12 13 14 15 16 17 19 20 21 22 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 41 42 43 44]
(45, 15)
[ 2  6 35 37 39] [ 0  1  3  4  5  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 36 38 40 41 42 43 44]
(45, 15)
[12 21 27 31 42] [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19 20 22 23 24 25
 26 28 29 30 32 33 34 35 36 37 38 39 40 41 43 44]
(45, 15)
[ 5 13 14 17 19] [ 0  1  2  3  4  6  7  8  9 10 11 12 15 16 18 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
(45, 15)
[ 1  4 26 43] [ 0  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44]
(45, 15)
[11 22 24 32] [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 23 25 26
 27 28 29 

In [25]:

#all_corr = []
from scipy.stats import pearsonr
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

def classification(training_set_x, testing_set_x, training_set_y, testing_set_y):
    
    train, y_train = separate_classes(training_set_x, training_set_y)
    
    n_train, n_test, n_ytrain, n_ytest = \
            train_test_split(train[1], y_train[1], test_size=.4, random_state=42)

    loss = 'cosine_similarity'
    activ='tanh'
    opt='adam'

    nodes = [500,250,50]

    create_autoencoder(n_train, n_train, n_test, n_test, 'RN', loss, activ, opt, nodes = nodes, epochs=50)

    encoder = pickle.load(open('RN_encoder', 'rb'))
    decoder = pickle.load(open('RN_decoder', 'rb'))

    enc = encoder.predict(testing_set_x)
    dec = decoder.predict(enc)

    enc_train = encoder.predict(training_set_x)
    dec_train = decoder.predict(enc_train)
    train_cl,test_cl = [],[]
    y_train_new = []
    y_test_new = []
    for d in range(len(dec_train)):
        corr_train = [pearsonr(dec_train[d][i:i+100],training_set_x[d][i:i+100])[0] for i in range(0,len(dec_train[d]),100)]
        if np.isfinite(corr_train).all():
            train_cl += [corr_train]
            y_train_new += [training_set_y[d]]

    for d in range(len(dec)):

        corr = [pearsonr(dec[d][i:i+100],testing_set_x[d][i:i+100])[0] for i in range(0,len(dec[d]),100)]

        if np.isfinite(corr).all():
            test_cl += [corr]
            y_test_new += [testing_set_y[d]]



    names = ["SVM", "RF","NB"]

    classifiers = [SVC(gamma=2, C=1, probability=True),
                   RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), GaussianNB()]
    

    _score = fus_classifier(classifiers, names, train_cl,\
                                  test_cl, np.array(y_train_new), np.array(y_test_new), show=False)
    return _score

all_score = []
for fsx in range(len(FS_X_train)):
    
    signal_train = FS_X_train[fsx]
    _ap_train = y_train[fsx]
    signal_test = FS_X_test[fsx]
    _ap_test = y_test[fsx]
    print(signal_train.shape)
    all_score += [classification(signal_train, signal_test, _ap_train, _ap_test)]
    
print(all_score)
df = pd.DataFrame(all_score)
df.to_csv (r'score_NB_bitalino.csv', index = None, header=True)


(600, 1000)

Autoencoder Created:
Layers: [500, 250, 50]
Input Length: 1000
Compression: 50
Activation: tanh
Optimizer: adam
Loss: cosine_similarity

Epoch 1/50
3/3 [==============================] - 0s 40ms/step - loss: -0.1704 - val_loss: -0.7673
Epoch 2/50
3/3 [==============================] - 0s 12ms/step - loss: -0.8161 - val_loss: -0.8835
Epoch 3/50
3/3 [==============================] - 0s 12ms/step - loss: -0.8962 - val_loss: -0.8989
Epoch 4/50
3/3 [==============================] - 0s 12ms/step - loss: -0.9064 - val_loss: -0.9003
Epoch 5/50
3/3 [==============================] - 0s 12ms/step - loss: -0.9074 - val_loss: -0.9004
Epoch 6/50
3/3 [==============================] - 0s 12ms/step - loss: -0.9075 - val_loss: -0.9005
Epoch 7/50
3/3 [==============================] - 0s 17ms/step - loss: -0.9075 - val_loss: -0.9006
Epoch 8/50
3/3 [==============================] - 0s 13ms/step - loss: -0.9077 - val_loss: -0.9008
Epoch 9/50
3/3 [==============================] - 0s 12ms/

In [28]:
table1 = pd.read_csv(r'score_NB_bitalino.csv', header=[0])
print(table1.mean())
print(table1.std())

0    75.412
dtype: float64
0    4.871754
dtype: float64


In [29]:

train, y_train = separate_classes(signal_train, _ap_train)

n_train, n_test, n_ytrain, n_ytest = \
        train_test_split(train[1], y_train[1], test_size=.4, random_state=42)


encoder = pickle.load(open('RN_encoder', 'rb'))
decoder = pickle.load(open('RN_decoder', 'rb'))

enc = encoder.predict(signal_test)
dec = decoder.predict(enc)

enc_train = encoder.predict(signal_train)
dec_train = decoder.predict(enc_train)
train_cl,test_cl = [],[]
y_train_new = []
y_test_new = []
for d in range(len(dec_train)):
    corr_train = [pearsonr(dec_train[d][i:i+100],signal_train[d][i:i+100])[0] for i in range(0,len(dec_train[d]),100)]
    if np.isfinite(corr_train).all():
        train_cl += [corr_train]
        y_train_new += [_ap_train[d]]

for d in range(len(dec)):

    corr = [pearsonr(dec[d][i:i+100],signal_test[d][i:i+100])[0] for i in range(0,len(dec[d]),100)]

    if np.isfinite(corr).all():
        test_cl += [corr]
        y_test_new += [_ap_test[d]]


ns = ["SVM", "RF", "NB"]

cl = [SVC(gamma=2, C=1, probability=True),
               RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), GaussianNB()]


all_scores = []
    
    # Train the classifier
y_pred = []

train_cl,test_cl, np.array(y_train_new), np.array(y_test_new)

cl[0].fit(train_cl, np.array(y_train_new).ravel())
cl[1].fit(train_cl, np.array(y_train_new).ravel())
cl[2].fit(train_cl, np.array(y_train_new).ravel())

proba1 = cl[0].predict_proba(test_cl)
proba2 = cl[1].predict_proba(test_cl)
proba3 = cl[1].predict_proba(test_cl)

#prob = [proba1[pb] if np.argmax([proba1[pb], proba2[pb]]) in [0,1] else proba2[pb] for pb in range(len(proba1))]
prob= [[np.mean([proba1[i,0],proba2[i,0],proba3[i,0]]),np.mean([proba1[i,1],proba2[i,1],proba3[i,1]])] for i in range(len(proba1))]


print(prob)
y_pred = [np.argmax(prob[i]) for i in range(len(prob))]

y_test = [1 if yt == 'N' else 0 for yt in y_test_new]
print(y_test, y_pred)
score = accuracy_score(y_test,y_pred)


#print(c.predict_proba(test))
# Get the classification accuracy
print(str(ns) + " --- Accuracy: " + str(score) + '%')
print('-----------------------------------------')
class_names = np.array(['Apnea', 'Normal'])

        

20/20 [==============================] - 0s 771us/step
[[0.273470965736019, 0.7265290342639812], [0.13920944320860903, 0.8607905567913909], [0.07870259838143319, 0.9212974016185669], [0.15755617963703802, 0.842443820362962], [0.2001417644275123, 0.7998582355724878], [0.2588069524309679, 0.7411930475690321], [0.20368063859454696, 0.7963193614054531], [0.4114689030688597, 0.5885310969311403], [0.20501163295851854, 0.7949883670414813], [0.28156834179133344, 0.7184316582086666], [0.569947946223693, 0.430052053776307], [0.5476772615001811, 0.4523227384998189], [0.6759844038648067, 0.3240155961351932], [0.5140673350335973, 0.4859326649664026], [0.3759339318825569, 0.6240660681174433], [0.21215072508897448, 0.7878492749110254], [0.5867859165851786, 0.41321408341482124], [0.15591603602228307, 0.844083963977717], [0.5077010216561431, 0.49229897834385694], [0.2983130091288124, 0.7016869908711877], [0.42053640386021357, 0.5794635961397864], [0.41030967480043473, 0.5896903251995651], [0.0959153407